In [1]:
import numpy as np
import pandas as pd
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
import datetime as dt
import pandas_datareader.data as web

start = dt.datetime(2019, 6, 15)
end = dt.datetime.now()

## Acquiring data from MOEX, Finam, Yahoo finance  

In [2]:
df_old=pd.read_excel('ML_DF_Regression.xlsx')
df_old.set_index('Date',inplace=True)

In [3]:
Brent = web.get_data_moex('BRQ9', start, end)
Brent.drop(['BOARDID', 'SECID', 'OPENPOSITIONVALUE', 'OPEN', 'LOW', 'HIGH', 'VALUE',  'OPENPOSITION','VOLUME'], 1, inplace=True)
Brent.index.name= 'Date'
Brent.sort_index(inplace=True) 
Brent.rename(columns={'CLOSE':'Brent'}, inplace=True)

EUR_USD = web.get_data_moex('EURUSD000TOM', start, end)
EUR_USD=EUR_USD.loc[EUR_USD.BOARDID=='CETS']
EUR_USD.drop(['BOARDID', 'SECID', 'SHORTNAME', 'VOLRUR', 'OPEN', 'LOW', 'HIGH', 'NUMTRADES',  'WAPRICE'], 1, inplace=True)
EUR_USD.index.name= 'Date'
EUR_USD.sort_index(inplace=True) 
EUR_USD.rename(columns={'CLOSE':'EUR_USD'}, inplace=True)

USD_RUB_New = web.get_data_moex('USD000UTSTOM', start, end)
USD_RUB_New=USD_RUB_New.loc[USD_RUB_New.BOARDID=='CETS']
USD_RUB_New.drop(['BOARDID', 'SECID', 'SHORTNAME',  'OPEN', 'WAPRICE'], 1, inplace=True)
USD_RUB_New.index.name= 'Date'
USD_RUB_New.rename(columns={'CLOSE':'USD_RUB','VOLRUR':'VOL_USD','LOW':'USD_LOW', 'HIGH':'USD_HIGH', 'NUMTRADES':'USD_NUMTRADES'}, inplace=True)
USD_RUB_New.sort_index(inplace=True) 

RGBITR_New= web.get_data_moex('RGBITR', start, end)
RGBITR_New.drop(['BOARDID', 'SECID', 'SHORTNAME', 'NAME', 'OPEN', 'LOW', 'HIGH','CLOSE','DECIMALS', 'CURRENCYID',  'DIVISOR','VALUE','DURATION','CAPITALIZATION'], 1, inplace=True)
RGBITR_New.index.name= 'Date'
RGBITR_New.index=pd.to_datetime(RGBITR_New.index)
RGBITR_New.rename(columns={'YIELD':'RGBITR_YIELD'}, inplace=True)
RGBITR_New.sort_index(inplace=True) 

USD_Repo = web.get_data_moex('MOEXREPOUSD', start, end)
USD_Repo.drop(['BOARDID', 'SECID', 'SHORTNAME', 'NAME', 'VALUE', 'OPEN', 'LOW', 'HIGH', 'DURATION', 'YIELD', 'DECIMALS', 'CAPITALIZATION', 'CURRENCYID', 'DIVISOR'], 1, inplace=True)
USD_Repo.rename(columns={'CLOSE':'USD_Repo'}, inplace=True)
USD_Repo.index.name= 'Date'
USD_Repo.index=pd.to_datetime(USD_Repo.index)
USD_Repo.sort_index(inplace=True) 

MOEXBC_New= web.get_data_moex('MOEXBC', start, end)
MOEXBC_New.drop(['BOARDID', 'SECID', 'SHORTNAME', 'NAME', 'OPEN', 'LOW', 'HIGH','DECIMALS', 'CURRENCYID',  'DIVISOR','VALUE','DURATION','CAPITALIZATION','YIELD'], 1, inplace=True)
MOEXBC_New.index.name= 'Date'
MOEXBC_New.index=pd.to_datetime(MOEXBC_New.index)
MOEXBC_New.rename(columns={'CLOSE':'MOEXBC'}, inplace=True)
MOEXBC_New.sort_index(inplace=True)

my_data=[ USD_RUB_New.USD_RUB, USD_RUB_New.VOL_USD, RGBITR_New.RGBITR_YIELD,EUR_USD.EUR_USD, USD_Repo.USD_Repo,MOEXBC_New.MOEXBC,Brent.Brent]
my_data=pd.concat(my_data,axis=1)
my_data.index=pd.to_datetime(my_data.index)
my_data.tail(3)

,USD_RUB,VOL_USD,RGBITR_YIELD,EUR_USD,USD_Repo,MOEXBC,Brent
Date,,,,,,,
2019-07-16,63.0450,1.450378e+11,7.34,1.12076,2.28,18677.56,66.66
2019-07-17,62.9350,1.139888e+11,7.34,1.12228,2.28,18413.87,64.18
2019-07-18,62.8075,1.206543e+11,7.35,1.12738,2.29,18337.25,61.79


In [4]:
tickers = ['RSX','^IRX','EEM']
ETFs = pd.DataFrame()    

for ticker in tickers:
        ETF = web.get_data_yahoo(ticker, start, end)
        ETF.rename(columns={'Adj Close': ticker}, inplace=True)
        ETF.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)
        if ETFs.empty:
            ETFs = ETF
        else:
            ETFs = ETFs.join(ETF, how='outer')
ETFs.rename(columns={'RSX': 'Russia_ETF', '^IRX':'Treas_4M','EEM': 'Emerging_Markets'}, inplace=True)

df_new=pd.merge(my_data,ETFs,on='Date')
df_new.sort_index(inplace=True)


In [5]:
main_df = df_old.append(df_new,sort=False)
main_df.index=pd.to_datetime(main_df.index)
main_df.sort_index(inplace=True)
main_df.drop_duplicates(subset='MOEXBC',inplace=True)
main_df.dropna(inplace=True)
main_df.tail(3)

,Brent,EUR_USD,Emerging_Markets,MOEXBC,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo,VOL_USD
Date,,,,,,,,,,
2019-07-16,66.66,1.12076,42.950001,18677.56,7.34,23.750000,2.103,63.0450,2.28,1.450378e+11
2019-07-17,64.18,1.12228,42.820000,18413.87,7.34,23.709999,2.073,62.9350,2.28,1.139888e+11
2019-07-18,61.79,1.12738,43.080002,18337.25,7.35,23.889999,2.013,62.8075,2.29,1.206543e+11


## Saving database to excel file

In [6]:
main_df.to_excel('ML_DF_Regression.xlsx')

## Reading back and dropping redundant features

In [7]:
main_df=pd.read_excel('ML_DF_Regression.xlsx')
main_df.drop(['VOL_USD','Emerging_Markets','MOEXBC'], 1,inplace=True)
main_df=main_df.tail(601)
main_df.reset_index(drop=True,inplace=True)
main_df.tail(3)

,Date,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
598,2019-07-16,66.66,1.12076,7.34,23.750000,2.103,63.0450,2.28
599,2019-07-17,64.18,1.12228,7.34,23.709999,2.073,62.9350,2.28
600,2019-07-18,61.79,1.12738,7.35,23.889999,2.013,62.8075,2.29


In [8]:
predict_df = main_df[(main_df.index == 600 )]
main_df.drop('Date', 1,inplace=True)

EPOCHS =250 # how many passes through our data
BATCH_SIZE =5  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.

# opt='adam'
opt='rmsprop'

NAME = f"{BATCH_SIZE}-{opt}--{main_df.columns.tolist()}--{int(time.time())}" 

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
filepath = "regression-{epoch:02d}-{val_mean_absolute_error:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_mean_absolute_error', verbose=1, save_best_only=True, mode='min')) # saves only the best ones

## Train/test split

In [9]:
test_db=main_df.loc[(main_df.index > 509) & (main_df.index < 600)]
test_db.tail(3)

,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
597,66.65,1.12556,7.33,23.930000,2.085,62.670,2.31
598,66.66,1.12076,7.34,23.750000,2.103,63.045,2.28
599,64.18,1.12228,7.34,23.709999,2.073,62.935,2.28


In [10]:
train_df = main_df[(main_df.index <= 509 )] 
train_df.tail(3)

,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
507,66.39,1.13756,8.26,20.420000,2.383,65.905,2.44
508,64.97,1.13635,8.25,20.440001,2.378,65.940,2.39
509,65.65,1.13350,8.24,20.500000,2.375,65.740,2.37


## Feature-wise normalization

In [11]:
train_targets = train_df.USD_RUB.values
train_data=train_df.drop("USD_RUB", 1).values

test_targets = test_db.USD_RUB.values
test_data=test_db.drop("USD_RUB", 1).values

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

## Building and training our network

In [16]:
model = Sequential()
model.add(Dense(128, activation='relu',input_shape=(train_data.shape[1],)))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(1))

model.compile(optimizer=opt, loss='mse', metrics=['mae'])

history = model.fit(
    train_data, train_targets,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(test_data, test_targets),
    callbacks=[tensorboard, checkpoint],
   )

Train on 510 samples, validate on 90 samples
Epoch 1/250
510/510 [==============================] - 7s 13ms/sample - loss: 3628.1911 - mean_absolute_error: 60.1855 - val_loss: 3844.5679 - val_mean_absolute_error: 62.0018
Epoch 2/250
510/510 [==============================] - 5s 10ms/sample - loss: 3326.5900 - mean_absolute_error: 57.6080 - val_loss: 3513.8215 - val_mean_absolute_error: 59.2731
Epoch 3/250
510/510 [==============================] - 2s 3ms/sample - loss: 2888.9164 - mean_absolute_error: 53.6627 - val_loss: 2978.1721 - val_mean_absolute_error: 54.5240
Epoch 4/250
510/510 [==============================] - 2s 3ms/sample - loss: 2353.5311 - mean_absolute_error: 48.3817 - val_loss: 2046.4826 - val_mean_absolute_error: 45.1775
Epoch 5/250
510/510 [==============================] - 1s 1ms/sample - loss: 1761.0005 - mean_absolute_error: 41.8007 - val_loss: 1637.0622 - val_mean_absolute_error: 40.3463
Epoch 6/250
510/510 [==============================] - 1s 1ms/sample - loss: 1

## Getting and normalizing features to predict on

In [17]:
predict_df

,Date,Brent,EUR_USD,RGBITR_YIELD,Russia_ETF,Treas_4M,USD_RUB,USD_Repo
600,2019-07-18,61.79,1.12738,7.35,23.889999,2.013,62.8075,2.29


In [18]:
predict_data=predict_df.drop(["USD_RUB",'Date'], 1).values
prediction_target=predict_df.USD_RUB.values
predict_data-= mean
predict_data /= std


## Load the most accurate model, ready to predict!

In [19]:
model = tf.keras.models.load_model("regression-222-0.591.model")

In [20]:
prediction = model.predict(test_data[:1],batch_size=1)

print("Real value =", prediction_target)
print("Prediction =", prediction) 

Real value = [62.8075]
Prediction = [[66.11786]]
